In [47]:
!pip install langchain langchain_openai chromadb pymupdf nest_asyncio --quiet

from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)
from langchain.schema.output_parser import StrOutputParser

!pip install PyMuPDF pytesseract Pillow --quiet
!pip install tokenizers
!pip install pymupdf

In [51]:
import pytesseract

print(pytesseract)

import chromadb
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader

<module 'pytesseract' from 'C:\\Users\\aswin\\anaconda3\\Lib\\site-packages\\pytesseract\\__init__.py'>


In [164]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

import fitz  # PyMuPDF
from PIL import Image
import pytesseract

# Define the path to the PDF file
pdf_path = r'C:\Users\aswin\OneDrive\Desktop\Important\Presentations\ML\Data\Form990\Central_Jersey_College_Prep_2022.pdf'

chromdb_path = './chroma.db'


import os
os.environ["OPENAI_API_KEY"] = 'sk-proj-ivq_WjkSlk5T3BlbkFJpzHw3wUQNnTJh-PTB1dc9yLb-unj9aOdikOj6PLW5fhUuky0CfMu91Gfp-kR3HjBN8BdoUuwIA'

# create vector store with Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores.utils import filter_complex_metadata # import filter_complex_metadata

# Open the PDF file
pdf_document = fitz.open(pdf_path)
pages=[]
# Iterate through each page
for page_num in range(pdf_document.page_count):
    zoom = 3.5
    mat = fitz.Matrix(zoom, zoom)
    page = pdf_document.load_page(page_num)
    pix = page.get_pixmap(matrix = mat)
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    # Perform OCR on the image
    text = pytesseract.image_to_string(img)
    pages.append(text)
    # Print the extracted text
    #print(f"Page {page_num + 1}:\n{text}\n")
    
# Close the PDF document
pdf_document.close()
#print(pages)
# split the pages into chunks
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=50
)
pages_to_persist = text_splitter.create_documents(pages)
pages.extend(pages_to_persist)
#chroma_setting = Settings(anonymized_telemetry=False, is_persistent=True, persist_directory=chromdb_path)
vectordb = Chroma.from_documents(documents=pages_to_persist, embedding=OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"]))#,persist_directory=chromdb_path,client_settings=chroma_setting)
vectordb.persist()
retriever = vectordb.as_retriever()

In [166]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [168]:
#Creating a RAG Pipeline
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
# RAG
template = """You are an AI language model Form990 Tax assistant.Answer the following question based on this context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
llm = ChatOpenAI(temperature=0, openai_api_key=os.environ["OPENAI_API_KEY"])
final_rag_chain = (
    #{"context": retriever | format_docs, "question": RunnablePassthrough()}

    RunnablePassthrough.assign(
        context=lambda x: format_docs(vectordb.similarity_search(x["question"], k=10)),
    )
    | prompt
    | llm
    | StrOutputParser()
)
    

In [178]:
question="In Part III Statement of Program Service Accomplishments Question 4a what is the Expenses including grants for Kingsford Highschool?"

In [180]:
final_rag_chain.invoke({"question":question})

'The Expenses including grants for Kingsford High School Athletic Booster Club in Part III, Line 4a is $45,036.'

In [182]:
questions = [
    "In Part IX Statement of Functional Expenses what is the Total Functional expenses for Management and general expenses for Kingsford High School and compare it with Central_Jersey_College?",
    "In Part VIII Statement of Revenue, what are all the other contributions gifts and grants for Central Jersey?",
    "In Part VIII Statement of Revenue, what are all the other contributions gifts and grants for Kingsford Highschool?",
    "In Part IX Statement of Functional Expenses what is the Total Office Expenses for Central Jersey?",
    "In Part III Statement of Program Service Accomplishments Question 4a what is the Expenses including grants for Central Jersey?",
    "In Part III Statement of Program Service Accomplishments Question 4a what is the Expenses including grants for Kingsford Highschool?"]
ground_truth = [
    "Kingsford High School is $45,036 and Central Jersey College Prep Charter School is $17,232,900.",
    "$724,773",
    "$3,368",
    "$401,845",
    "$17,232,900",
    "$45,036"
    ]

In [184]:
!pip install datasets --quiet
from datasets import Dataset

In [186]:
answers  = []
contexts = []

# traversing each question and passing into the chain to get answer from the system
for question in questions:
    answers.append(final_rag_chain.invoke({"question":question}))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(question)])

# Preparing the dataset
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truth
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

dataset.to_pandas()

,question,answer,contexts,ground_truth
0,In Part IX Statement of Functional Expenses wh...,The Total Functional expenses for Management a...,[Part | Summary\n1 Briefly describe the organi...,"Kingsford High School is $45,036 and Central J..."
1,"In Part VIII Statement of Revenue, what are al...","In Part VIII Statement of Revenue, the total a...",[efile GRAPHIC print - DO NOT PROCESS | As Fil...,"$724,773"
2,"In Part VIII Statement of Revenue, what are al...","In Part VIII Statement of Revenue, the ""All ot...",[efile GRAPHIC print - DO NOT PROCESS | As Fil...,"$3,368"
3,In Part IX Statement of Functional Expenses wh...,The Total Office Expenses for Central Jersey C...,[Part | Summary\n1 Briefly describe the organi...,"$401,845"
4,In Part III Statement of Program Service Accom...,For Central Jersey College Prep Charter School...,[Form 990 (2022) Page 2\nPart Ill Statement of...,"$17,232,900"
5,In Part III Statement of Program Service Accom...,The Expenses including grants for Kingsford Hi...,[Form 990 (2022) Page 2\nPart Ill Statement of...,"$45,036"


In [188]:
!pip install ragas --quiet
import ragas

In [189]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset=dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

df = result.to_pandas()
df

Evaluating:   0%|          | 0/24 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy
0,In Part IX Statement of Functional Expenses wh...,[Part | Summary\n1 Briefly describe the organi...,The Total Functional expenses for Management a...,"Kingsford High School is $45,036 and Central J...",0.000000,0.0,0.5,0.000000
1,"In Part VIII Statement of Revenue, what are al...",[efile GRAPHIC print - DO NOT PROCESS | As Fil...,"In Part VIII Statement of Revenue, the total a...","$724,773",0.000000,0.0,0.0,0.933359
2,"In Part VIII Statement of Revenue, what are al...",[efile GRAPHIC print - DO NOT PROCESS | As Fil...,"In Part VIII Statement of Revenue, the ""All ot...","$3,368",0.000000,0.0,0.0,0.943600
3,In Part IX Statement of Functional Expenses wh...,[Part | Summary\n1 Briefly describe the organi...,The Total Office Expenses for Central Jersey C...,"$401,845",0.000000,0.0,0.0,0.906727
4,In Part III Statement of Program Service Accom...,[Form 990 (2022) Page 2\nPart Ill Statement of...,For Central Jersey College Prep Charter School...,"$17,232,900",1.000000,1.0,0.0,0.877405
5,In Part III Statement of Program Service Accom...,[Form 990 (2022) Page 2\nPart Ill Statement of...,The Expenses including grants for Kingsford Hi...,"$45,036",0.916667,1.0,1.0,0.905386
